In [1]:
#%load_ext autoreload
#%autoreload 2
import os
import lightkurve as lk
import pandas as pd
import numpy as np
import dynesty
from granulation_fitting import granulation_fit 
from matplotlib.pyplot import *
rcParams['font.size'] = 18

/home/nielsemb/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
download_dir = '/home/nielsemb/work/mounts/Bluebear_data/data'

workDir = '/home/nielsemb/work/repos/granulation'

prior_data = pd.read_csv(os.path.join(*[workDir, 'prior_data.csv']))

In [3]:
test_numaxs = np.linspace(min(prior_data['numax']), max(prior_data['numax']), 10)
idxs = np.array([np.argmin(abs(prior_data['numax'].values - nu)) for nu in test_numaxs])

In [ ]:
cornerN = 5000

figM, axM = subplots(figsize=(16,9))
 
i = 13441

ID = prior_data.loc[i, 'ID']
print(ID)

outputDir = os.path.join(*[workDir, 'results', ID])

if not os.path.exists(outputDir):
    os.makedirs(outputDir)

_numax = prior_data.loc[i, 'numax']

numax = [10**_numax, 0.1*10**_numax]

gfit = granulation_fit(ID, numax, download_dir, pcadim=8)

sampler, samples = gfit.runDynesty()

gfit.storeResults(outputDir)

gfit.plotModel(figM, axM, outputDir=outputDir);
axM.clear()

gfit.plotModel(figM, axM, samples, outputDir=outputDir);
axM.clear()

TIC191929687
Using cached search result.
Loading fits files from cache.
Binning time series by 3


4402it [01:31, 41.14it/s, bound: 122 | nc: 32 | ncall: 124602 | eff(%):  3.533 | loglstar:   -inf < -40204.685 <    inf | logz: -40226.180 +/-  0.315 | dlogz:  3.602 >  0.209]

In [ ]:
gfit.samples = np.random.normal(0, 1, (1000, 12))

In [ ]:
import dill

In [ ]:
dill.dump(gfit, open("test.p", "wb"))

In [ ]:
gfit0 = granulation_fit(ID, numax, download_dir, pcadim=8)

figM, axM = subplots(figsize=(16,9))

gfit0.plotModel(figM, axM)

In [ ]:
def wfunc(self, n=1):
     
    ppf, pdf = self.getQuantileFuncs(self.data_F[:, :1])

    w = 1/pdf[0](self.data_F[:, 0])**n
       
    return w

fig, ax = subplots()
ax.axhline(10**_numax)
for k in range(0, 20, 4):
    gfit = granulation_fit(ID, numax, download_dir, pcadim=3, weights=wfunc, weight_args={'n':k})

    mu_pr = np.zeros(gfit.ndim) + 0.5

    theta = gfit.ptform(mu_pr)

    params = gfit.unpackParams(theta)

    ax.plot(k, params[0], 'o')
    ax.plot(k, 10**gfit.DR.mu[0], 'o', color='k')

In [ ]:

# figM, axM = subplots(figsize=(16,9))
# gfit1.plotModel(figM, axM);

figM, axM = subplots(figsize=(16,9))
gfit0.plotModel(figM, axM, samples0);
figM, axM = subplots(figsize=(16,9))
gfit1.plotModel(figM, axM, samples1);

In [ ]:
fig, ax = subplots(gfit0.ndim, gfit0.ndim, figsize=(20,20))
gfit0.makeSamplingCorner(fig, samples0)

In [ ]:
fig, ax = subplots(gfit1.ndim, gfit1.ndim, figsize=(20,20))
gfit1.makeSamplingCorner(fig, samples1)

In [ ]:
gfit.DR.makeTrainingCorner()

In [ ]:
gfit.DR.makeDRTrainingCorner()

In [ ]:
fig, ax = subplots(gfit.ndim, gfit.ndim, figsize=(20,20))
gfit.makePriorCorner(fig)

In [ ]:
fig, ax = subplots(len(gfit.labels), len(gfit.labels), figsize=(30,30))
gfit.makeFullCorner(fig, samples, N=500)    

In [ ]:
gfit = granulation_fit(ID, numax, download_dir, pcadim=None)

In [ ]:
figM, axM = subplots(figsize=(16,9))
gfit.plotModel(figM, axM, outputDir=outputDir);



In [ ]:
sampler, samples = gfit.runDynesty()

In [ ]:
figM, axM = subplots(figsize=(16,9))
gfit.plotModel(figM, axM, samples, outputDir=outputDir);

In [ ]:
fig, ax = subplots(len(gfit.labels), len(gfit.labels), figsize=(30,30))
gfit.makeSamplingCorner(fig, samples)

In [ ]:
def wfunc(self, n=1):
     
    ppf, pdf = self.getQuantileFuncs(self.data_F[:, :1])

    w = 1/pdf[0](self.data_F[:, 0])**n
       
    return w

from DR import PCA

import utils

_numax = prior_data.loc[13441, 'numax']

numax = [10**_numax, 0.1*10**_numax]

numax = utils.to_log10(*numax)          

pcalabels = ['bkg_numax', 'bkg_envHeight', 'bkg_envWidth',
             'H1_power', 'H1_nu', 'H1_exp',
             'H2_power', 'H2_nu', 'H2_exp',] 

P_test = PCA(numax, pcalabels, weights=wfunc, weight_args={'n':2}, N=1000)

# P_test.fit_weightedPCA(2)

In [ ]:


P_test.weights

In [ ]:
idxsort = np.array(np.argsort(P_test.data_F[:, 0]))

hist(P_test.data_F[:, 0])
plot(P_test.data_F[:, 0][idxsort], P_test.weights[idxsort])
axvline(numax[0])

In [ ]:
P_test.data_F[:, 0][idxsort]

In [ ]:
y0 = np.random.normal(loc= 0, scale=1, size=1000)
y1 = np.random.normal(loc=10, scale=1, size=1000)
y = np.append(y0, y1)

In [ ]:
w = np.zeros_like(y)+1e-20
w[y>5] = 1

In [ ]:
np.cov(y, rowvar=False, aweights=w)

In [ ]:
np.cov(y, rowvar=False)

In [ ]:
np.cov(y[y>5])

In [ ]:
semilogy(gfit1.DR.data_F[:, 0], gfit1.DR.weights**2, '.')

In [ ]:

np.cov(gfit1.DR.data_F[:, 0], rowvar=False, aweights=gfit1.DR.weights)

In [ ]:
_X = gfit1.DR.scale(gfit1.DR.data_F)


In [ ]:
np.cov(_X[:, 0], rowvar=False, aweights=gfit1.DR.weights)

In [ ]:
import statsmodels.api as sm
import scipy.stats as st
import scipy.integrate as si
import jax.numpy as jnp
import utils

y = gfit.DR.data_F[:, :1]
print(y.shape)
ppfs, pdfs = gfit.DR.getQuantileFuncs(y)
 
x = np.linspace(ppfs[0](0), ppfs[0](1), 100)
plot(x, pdfs[0](x))
hist(y[:, 0], bins =30, density=True)

In [ ]:

# kde = sm.nonparametric.KDEUnivariate(np.array(y).real)

# kde.fit(adjust=1, cut=4)

# print(kde.support.min(), kde.support.max())

# Q = kde.icdf
# A = jnp.linspace(0, 1, len(Q))
# ppf = utils.jaxInterp1D(A, Q)
# pdf = kde.evaluate


# fig, ax = subplots(1,2, figsize=(12,6))
# #ax[0].hist(y, bins = 30, density=True)

# x = np.linspace(ppf(0), ppf(1), 100)
# #ax[0].plot(kde.support, kde.evaluate(kde.support))
# #ax[0].plot(x, pdf(x))


# ax[1].plot(A, Q)

# Q0 = utils.getCurvePercentiles(kde.support, 
#                                kde.evaluate(kde.support),
#                                percentiles=np.linspace(0, 1, len(kde.cdf)))

# A0 = jnp.linspace(0, 1, len(Q0))
# ppf0 = utils.jaxInterp1D(A0, Q0)
# pdf0 = kde.evaluate

# 
# ax[0].plot(x0, np.log(pdf0(x0)), alpha = 0.5, lw=5)

# #ax[0].set_xlim(2.6, 2.7)
# #ax[0].set_ylim(0, 0.5)
# # ax[0].plot(kde1.support, kde1.cdf)

# # x = np.linspace(0, 1, len(kde1.cdf))
# # ax[1].plot(x, kde1.icdf)




# # ppf = 
# # ax[1].plot(x, ppf)




# # A = jnp.linspace(0, 1, len(Q))

# # ppf = utils.jaxInterp1D(A, Q)

# # pdf = kde.evaluate

# # hist(y, density=True)
# # x = np.linspace(ppf(0)-1, ppf(1)+1)
# # plot(x, pdf(x))


In [ ]:
G = utils.normal(mu=0, sigma=1)
y = G.ppf(np.random.uniform(0, 1, 100000))

In [ ]:
x = np.linspace(G.ppf(1e-9), G.ppf(1-1e-9), 10000)

In [ ]:
y_ = utils.getCurvePercentiles(x, G.pdf(x), percentiles = np.linspace(0, 1, 10000))

In [ ]:
#hist(y, density=True)
plot(np.linspace(0, 1, 10000), y_)

In [ ]:
len(a), len(kde1.support), len(np.linspace(0,1,len(kde1.cdf)))

In [ ]:
plot(ppf)

In [ ]:
cSumNorm = si.cumtrapz(a, kde1.support, initial=0) / np.trapz(a, kde1.support)
percentiles = np.linspace(0,1,len(kde1.cdf))
#print(percentiles)
#percs = np.array([x[cSumNorm >= p][0] for p in percentiles])

In [ ]:
for p in percentiles:
    print(p, len(x[kde1.cdf >= p]))

In [ ]:
max(kde1.cdf)

In [ ]:
plot(kde1.support, a)

In [ ]:
kde.support

In [ ]:
len(kde.cdf)

In [ ]:
import scipy.stats as st

s = st.norm(loc= 0, scale=1)

In [ ]:
a = np.linspace(0, 1, 100)
plot(a, s.ppf(a))

In [ ]:
_X = gfit.DR.scale(gfit.DR.data_F)
         
W = jnp.diag(gfit.DR.weights)

C = _X.T@W@_X * jnp.sum(gfit.DR.weights) / (jnp.sum(gfit.DR.weights)**2 - jnp.sum(gfit.DR.weights**2))
C = np.array(C)
eigvals, eigvectors = jnp.linalg.eig(C)

In [ ]:
eigvectors

In [ ]:
for i in range(C.shape[0]):
    C[i, i] *= 50

In [ ]:
eigvals, eigvectors = jnp.linalg.eig(C)

In [ ]:
eigvectors

In [ ]:
import utils
import scipy.integrate as si

y = gfit.DR.data_F[:, 0]

miny = y.min()
maxy = y.max()



_dy = (ppf(0.01)-ppf(0))/1000
_y = y.min() - _dy * np.arange(1, 11)**2

_cdf = si.cumtrapz(pdf(_y[::-1]), _y[::-1]) 

    
    

    

In [ ]:
G = utils.normal(mu=0, sigma=1)

In [ ]:
x = np.linspace(-10, 10, 100)
y = G.pdf(x)

In [ ]:

c = si.cumtrapz(y, x, initial=0)

In [ ]:
plot(x, c)

In [ ]:
kde.kernel

In [ ]:
print(kde.kernel.domain)

In [ ]:
np.allclose(kde.support, y)